# Demos

This is the notebook containing the demos for Feature Store, Model Monitor, and Clarify. Testing for these exercises was performed using __2 vCPU + 4 GiB notebook instance with Python 3 (TensorFlow 2.1 Python 3.6 CPU Optimized) kernel__.

## Staging

We'll begin by initializing some variables that are used throughout the demos. These are often assumed to be present in code samples you'll find in the AWS documenation.

In [1]:
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role

role = get_execution_role()
session = sagemaker.Session()
region = session.boto_region_name
bucket = session.default_bucket()

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Feature Store
---

Feature store is a special database to give ML systems a consistent data flow across training and inference workloads. It can ingest data in batches (for training) as well as serve input features to models with very low latency for real-time prediction.

For this demo we'll use the boston housing dataset, which you can learn more about here: https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [2]:
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data(test_split=0.1, seed=1234)

# Manually add headers
train_headers = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"]
test_headers = ["MEDV"]

2025-03-09 05:42:27.536011: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[03/09/25 05:42:34] INFO     generated new fontManager                                         ]8;id=284314;file:///opt/conda/lib/python3.11/site-packages/matplotlib/font_manager.py\font_manager.py]8;;\:]8;id=708833;file:///opt/conda/lib/python3.11/site-packages/matplotlib/font_manager.py#1639\1639]8;;\

57026/57026 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
region

'us-east-1'

In [4]:
session

In [5]:
role

'arn:aws:iam::620425758198:role/service-role/AmazonSageMaker-ExecutionRole-20250307T190825'

In [6]:
bucket

'sagemaker-us-east-1-620425758198'

In [7]:
import pandas as pd
import time
import uuid


boston_train = pd.DataFrame(x_train, columns=train_headers)

Once we have our data, we can create a feature group. Remember to attach event time and ID columns - Feature Store needs them.

In [8]:
boston_train["EventTime"] = time.time()
boston_train["id"] = range(len(boston_train))

# Create feature group
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group = FeatureGroup(
    name="boston-features", sagemaker_session=session
)

# Load Feature definitions
feature_group.load_feature_definitions(data_frame=boston_train)

[FeatureDefinition(feature_name='CRIM', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='ZN', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='INDUS', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='CHAS', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='NOX', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='RM', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='AGE', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='DIS', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='RAD', feature

The feature group is not created until we call the `create` method, let's do that now:

In [9]:
feature_group.create(
    s3_uri=f"s3://{bucket}/features",
    record_identifier_name='id',
    event_time_feature_name="EventTime",
    role_arn=role,
    enable_online_store=True,
)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:620425758198:feature-group/boston-features',
 'ResponseMetadata': {'RequestId': 'fb6ac9d9-ee90-49a3-bd30-f8268626a04e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fb6ac9d9-ee90-49a3-bd30-f8268626a04e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '92',
   'date': 'Sun, 09 Mar 2025 05:48:07 GMT'},
  'RetryAttempts': 0}}

For applications, we can create a lightweight client to retrieve data with low latency:

In [10]:
runtime = session.boto_session.client(
  'sagemaker-featurestore-runtime',
  region_name=region
)

data = runtime.get_record(
    FeatureGroupName="boston-features",
    RecordIdentifierValueAsString="0"
)

If we try to get records before we ingest any data, the response comes back empty:

In [11]:
data

{'ResponseMetadata': {'RequestId': '5238b526-980b-4567-b240-20b595e427f4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5238b526-980b-4567-b240-20b595e427f4',
   'content-type': 'application/json',
   'content-length': '32',
   'date': 'Sun, 09 Mar 2025 05:49:03 GMT'},
  'RetryAttempts': 0}}

In [12]:
feature_group.ingest(data_frame=boston_train, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='boston-features', feature_definitions={'CRIM': {'FeatureName': 'CRIM', 'FeatureType': 'Fractional'}, 'ZN': {'FeatureName': 'ZN', 'FeatureType': 'Fractional'}, 'INDUS': {'FeatureName': 'INDUS', 'FeatureType': 'Fractional'}, 'CHAS': {'FeatureName': 'CHAS', 'FeatureType': 'Fractional'}, 'NOX': {'FeatureName': 'NOX', 'FeatureType': 'Fractional'}, 'RM': {'FeatureName': 'RM', 'FeatureType': 'Fractional'}, 'AGE': {'FeatureName': 'AGE', 'FeatureType': 'Fractional'}, 'DIS': {'FeatureName': 'DIS', 'FeatureType': 'Fractional'}, 'RAD': {'FeatureName': 'RAD', 'FeatureType': 'Fractional'}, 'TAX': {'FeatureName': 'TAX', 'FeatureType': 'Fractional'}, 'PTRATIO': {'FeatureName': 'PTRATIO', 'FeatureType': 'Fractional'}, 'B': {'FeatureName': 'B', 'FeatureType': 'Fractional'}, 'LSTAT': {'FeatureName': 'LSTAT', 'FeatureType': 'Fractional'}, 'EventTime': {'FeatureName': 'EventTime', 'FeatureType': 'Fractional'}, 'id': {'FeatureName': 'id', 'FeatureType': 'Integral'}

In [13]:
data = runtime.get_record(
    FeatureGroupName="boston-features",
    RecordIdentifierValueAsString="0"
)
data

{'ResponseMetadata': {'RequestId': '02c3c465-106b-41bb-8d36-ec0d84a5a05a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '02c3c465-106b-41bb-8d36-ec0d84a5a05a',
   'content-type': 'application/json',
   'content-length': '1110',
   'date': 'Sun, 09 Mar 2025 05:49:41 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'CRIM', 'ValueAsString': '0.01951'},
  {'FeatureName': 'ZN', 'ValueAsString': '17.5'},
  {'FeatureName': 'INDUS', 'ValueAsString': '1.38'},
  {'FeatureName': 'CHAS', 'ValueAsString': '0.0'},
  {'FeatureName': 'NOX', 'ValueAsString': '0.4161'},
  {'FeatureName': 'RM', 'ValueAsString': '7.104'},
  {'FeatureName': 'AGE', 'ValueAsString': '59.5'},
  {'FeatureName': 'DIS', 'ValueAsString': '9.2229'},
  {'FeatureName': 'RAD', 'ValueAsString': '3.0'},
  {'FeatureName': 'TAX', 'ValueAsString': '216.0'},
  {'FeatureName': 'PTRATIO', 'ValueAsString': '18.6'},
  {'FeatureName': 'B', 'ValueAsString': '393.24'},
  {'FeatureName': 'LSTAT', 'ValueAsString': '8.05'}

## Model Monitor

In this demo we create a monitoring schedule for a deployed model. We'll begin by reloading our data from the previous demo.

In [14]:
from tensorflow.keras.datasets import boston_housing
import pandas as pd

(x_train, y_train), (x_test, y_test) = boston_housing.load_data(test_split=0.1, seed=1234)
headers = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"]


train = pd.DataFrame(x_train, columns=headers)
train["MEDV"] = y_train

# Target variable must come first per https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html
train.set_index(train.pop('MEDV'), inplace=True)
train.reset_index(inplace=True)
train

,MEDV,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,33.0,0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
1,27.5,0.14866,0.0,8.56,0.0,0.5200,6.727,79.9,2.7778,5.0,384.0,20.9,394.76,9.42
2,5.6,25.04610,0.0,18.10,0.0,0.6930,5.987,100.0,1.5888,24.0,666.0,20.2,396.90,26.77
3,21.2,3.67367,0.0,18.10,0.0,0.5830,6.312,51.9,3.9917,24.0,666.0,20.2,388.62,10.58
4,14.9,9.51363,0.0,18.10,0.0,0.7130,6.728,94.1,2.4961,24.0,666.0,20.2,6.68,18.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,17.9,18.81100,0.0,18.10,0.0,0.5970,4.628,100.0,1.5539,24.0,666.0,20.2,28.79,34.37
451,14.5,8.49213,0.0,18.10,0.0,0.5840,6.348,86.1,2.0527,24.0,666.0,20.2,83.45,17.64
452,12.7,4.66883,0.0,18.10,0.0,0.7130,5.976,87.9,2.5806,24.0,666.0,20.2,10.48,19.01
453,17.8,0.31827,0.0,9.90,0.0,0.5440,5.914,83.2,3.9986,4.0,304.0,18.4,390.70,18.33


In [15]:
test =  pd.DataFrame(x_test)
test["MEDV"] = y_test

# Target variable must come first per https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html
test.set_index(test.pop('MEDV'), inplace=True)
test.reset_index(inplace=True)

Now we'll upload the data to S3 as train and validation data, then train a model:

In [16]:
train.to_csv("train.csv", header=False, index=False)
test.to_csv("validation.csv", header=False, index=False)

val_location = session.upload_data('./validation.csv', key_prefix="data")
train_location = session.upload_data('./train.csv', key_prefix="data")

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

In [17]:
from sagemaker.model_monitor import DataCaptureConfig

algo_image = sagemaker.image_uris.retrieve("xgboost", region, version='latest')
s3_output_location = f"s3://{bucket}/models/boston_model"

model=sagemaker.estimator.Estimator(
    image_uri=algo_image,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session()
)

model.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='reg:linear',
                        early_stopping_rounds=10,
                        num_round=200)


model.fit({'train': s3_input_train, 'validation': s3_input_validation})

[03/09/25 05:51:00] INFO     Ignoring unnecessary instance type: None.                            ]8;id=475060;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=571750;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#530\530]8;;\

[03/09/25 05:51:01] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=920169;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=401082;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: xgboost-2025-03-09-05-51-01-627       ]8;id=372536;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=636498;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-03-09 05:51:02 Starting - Starting the training job...
..25-03-09 05:51:26 Starting - Preparing the instances for training.
.....03-09 05:52:02 Downloading - Downloading input data.
....Arguments: trainng - Downloading the training image.
[2025-03-09:05:53:49:INFO] Running standalone xgboost training.
[2025-03-09:05:53:49:INFO] File size need to be processed in the node: 0.04mb. Available memory size in the node: 8565.78mb
[2025-03-09:05:53:49:INFO] Determined delimiter of CSV input is ','
[05:53:49] S3DistributionType set as FullyReplicated
[05:53:49] 455x13 matrix with 5915 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-03-09:05:53:49:INFO] Determined delimiter of CSV input is ','
[05:53:49] S3DistributionType set as FullyReplicated
[05:53:49] 51x13 matrix with 663 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[05:53:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 p

Now that the training job has finished, we can configure a deployment for data capture, then deploy:

In [18]:
capture_uri = f's3://{bucket}/data-capture'

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=capture_uri
)

In [19]:
xgb_predictor = model.deploy(
    initial_instance_count=1, instance_type='ml.m4.xlarge',
    data_capture_config=data_capture_config
)

[03/09/25 05:54:19] INFO     Creating model with name: xgboost-2025-03-09-05-54-19-132              ]8;id=593957;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=292254;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name xgboost-2025-03-09-05-54-19-132     ]8;id=920904;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=685031;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#5889\5889]8;;\

[03/09/25 05:54:20] INFO     Creating endpoint with name xgboost-2025-03-09-05-54-19-132            ]8;id=687339;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=334397;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4711\4711]8;;\

-------!

Here we can provide some sample code to test the deployed model:

In [20]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [21]:
inputs = test.copy()
inputs = inputs.drop(columns=inputs.columns[0])

x_pred = xgb_predictor.predict(inputs.sample(5).values).decode('utf-8')

In [22]:
s3_input_train

We define the Model Monitor and suggest a baseline:

In [23]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

[03/09/25 05:59:26] INFO     Ignoring unnecessary instance type: None.                            ]8;id=647265;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=361752;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#530\530]8;;\

In [24]:
my_monitor.suggest_baseline(
    baseline_dataset=f's3://{bucket}/data/train.csv',
    dataset_format=DatasetFormat.csv(header=False),
)

[03/09/25 05:59:30] INFO     Creating processing-job with name                                      ]8;id=268656;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=76590;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1575\1575]8;;\
                             baseline-suggestion-job-2025-03-09-05-59-30-482                                       

...........2025-03-09 06:01:22.115464: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-09 06:01:22.115496: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-03-09 06:01:23.782304: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-03-09 06:01:23.782335: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-03-09 06:01:23.782360: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-2-231-158.ec2.internal): /proc/driver/nvidia/version does not exist
2025-03-09 06:01:23.782635: I tensorf

Lastly, the Model Monitor must be scheduled, or it won't actually run regular processing jobs on the captured data:

In [25]:
from sagemaker.model_monitor import CronExpressionGenerator

my_monitor.create_monitoring_schedule(
    monitor_schedule_name='my-monitoring-schedule',
    endpoint_input=xgb_predictor.endpoint_name,
    statistics=my_monitor.baseline_statistics(),
    constraints=my_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)

[03/09/25 06:04:55] INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=483614;file:///opt/conda/lib/python3.11/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=94431;file:///opt/conda/lib/python3.11/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

[03/09/25 06:04:56] INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=818243;file:///opt/conda/lib/python3.11/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=174866;file:///opt/conda/lib/python3.11/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

                    INFO     Creating Monitoring Schedule with name:                       ]8;id=768494;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/model_monitoring.py\model_monitoring.py]8;;\:]8;id=294724;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/model_monitoring.py#1560\1560]8;;\
                             my-monitoring-schedule                                                                

## Clarify

This Clarify demo builds on the previous demo: we follow the same pattern of define-configure-schedule for our Monitor. Clarify, however, needs more config. We define `SHAPConfig`, `ModelConfig`, `ExplainabilityAnalysisConfig`, and pass them all to the scheduling method.

In [23]:
model_explainability_monitor = sagemaker.model_monitor.ModelExplainabilityMonitor(
    role=role,
    sagemaker_session=session,
    max_runtime_in_seconds=1800,
)


shap_config = sagemaker.clarify.SHAPConfig(
    baseline=[train.mean().astype(int).to_list()[1:]],
    num_samples=int(x_train.size),
    agg_method="mean_abs",
    save_local_shap_values=False,
)


model_config = sagemaker.clarify.ModelConfig(
    model_name="xgboost-2021-08-25-15-19-33-499",
    instance_count=1,
    instance_type='ml.m4.xlarge',
    content_type="text/csv",
    accept_type="text/csv",
)

analysis_config = sagemaker.model_monitor.ExplainabilityAnalysisConfig(
        explainability_config=shap_config,
        model_config=model_config,
        headers=train.columns.to_list()[1:],
    )

explainability_uri = f"s3://{bucket}/model_explainability"
model_explainability_monitor.create_monitoring_schedule(
    output_s3_uri=explainability_uri,
    analysis_config=analysis_config,
    endpoint_input=xgb_predictor.endpoint_name,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)

In [25]:
model_explainability_monitor.delete_monitoring_schedule()


Deleting Monitoring Schedule with name: monitoring-schedule-2021-09-02-22-00-36-508


In [32]:
my_monitor.delete_monitoring_schedule()


Deleting Monitoring Schedule with name: my-monitoring-schedule
